# Neural Network

In [125]:
#imports
from __future__ import division
from collections import *
import math, random
import matplotlib
import matplotlib.pyplot as plt

In [126]:
def dot(v,w):
    return sum([v_i*w_i for v_i,w_i in zip(v,w)])

In [127]:
def sigmoid(x):
    return 1/(1 + math.exp(-x))
def neuron_output(weights, inputs):
    return sigmoid(dot(weights,inputs))


## Feed-forward

In [128]:

def feed_forward(neural_network, input_vector):
    """takes in a neural network (represented as a list of lists of lists of weights)
    and returns the output from forward-propagating the input"""

    outputs = []

    for layer in neural_network:

        input_with_bias = input_vector + [1]             # add a bias input
        output = [neuron_output(neuron, input_with_bias) # compute the output
                  for neuron in layer]                   # for this layer
        outputs.append(output)                           # and remember it

        # the input to the next layer is the output of this one
        input_vector = output

    return outputs

In [129]:
xor_network = [[[20, 20, -30],
                [20, 20, -10]],
                [[-60, 60, -30]]]
for x in [0,1]:
    for y in [0,1]:
        print(x,"XOR",y," => ",feed_forward(xor_network,[x,y])[-1][0])

0 XOR 0  =>  9.38314668300676e-14
0 XOR 1  =>  0.9999999999999059
1 XOR 0  =>  0.9999999999999059
1 XOR 1  =>  9.383146683006828e-14


## Back-propagation

In [130]:
def backpropagate(network, input_vector, target):

    hidden_outputs, outputs = feed_forward(network, input_vector)

    # the output * (1 - output) is from the derivative of sigmoid
    output_deltas = [output * (1 - output) * (output - target[i])
                     for i, output in enumerate(outputs)]

    # adjust weights for output layer (network[-1])
    for i, output_neuron in enumerate(network[-1]):
        for j, hidden_output in enumerate(hidden_outputs + [1]):
            output_neuron[j] -= output_deltas[i] * hidden_output

    # back-propagate errors to hidden layer
    hidden_deltas = [hidden_output * (1 - hidden_output) *
                      dot(output_deltas, [n[i] for n in network[-1]])
                     for i, hidden_output in enumerate(hidden_outputs)]

    # adjust weights for hidden layer (network[0])
    for i, hidden_neuron in enumerate(network[0]):
        for j, input in enumerate(input_vector + [1]):
            hidden_neuron[j] -= hidden_deltas[i] * input

In [131]:
raw_digits = [
          """11111
             1...1
             1...1
             1...1
             11111""",

          """..1..
             ..1..
             ..1..
             ..1..
             ..1..""",

          """11111
             ....1
             11111
             1....
             11111""",

          """11111
             ....1
             11111
             ....1
             11111""",

          """1...1
             1...1
             11111
             ....1
             ....1""",

          """11111
             1....
             11111
             ....1
             11111""",

          """11111
             1....
             11111
             1...1
             11111""",

          """11111
             ....1
             ....1
             ....1
             ....1""",

          """11111
             1...1
             11111
             1...1
             11111""",

          """11111
             1...1
             11111
             ....1
             11111"""]

def make_digit(raw_digit):
    return [1 if c == '1' else 0
            for row in raw_digit.split("\n")
            for c in row.strip()]

inputs = list(map(make_digit, raw_digits))
targets = [[1 if i==j else 0 for i in range(10)]
          for j in range(10)]

In [132]:
random.seed(0)
input_size = 25
num_hidden = 5
output_size = 10
hidden_layer = [[random.random() for _ in range(input_size +1)]
               for _ in range(num_hidden)]

output_layer = [[random.random() for _ in range(num_hidden + 1)]
               for _ in range(output_size)]
network = [hidden_layer,output_layer]

### Train and predict

In [133]:
def predict(input):
    return feed_forward(network,input)[-1].index(max(feed_forward(network,input)[-1]))
# predict(inputs[7])
def print_predict():
    print("--------------------------------------------------------------------")
    for i in range(10):
        print("True => ",i,"\t Predict => ",predict(inputs[i]))
    print("=======================================================================")

In [134]:
for i in range(10000):
    for input_vec, target_vec in zip(inputs,targets):
        backpropagate(network,input_vec,target_vec)
    if i%1000 == 0:
        print("Predicting for ",i)
        print_predict()

Predicting for  0
--------------------------------------------------------------------
True =>  0 	 Predict =>  8
True =>  1 	 Predict =>  8
True =>  2 	 Predict =>  8
True =>  3 	 Predict =>  8
True =>  4 	 Predict =>  8
True =>  5 	 Predict =>  8
True =>  6 	 Predict =>  8
True =>  7 	 Predict =>  8
True =>  8 	 Predict =>  8
True =>  9 	 Predict =>  8
Predicting for  1000
--------------------------------------------------------------------
True =>  0 	 Predict =>  9
True =>  1 	 Predict =>  1
True =>  2 	 Predict =>  9
True =>  3 	 Predict =>  9
True =>  4 	 Predict =>  9
True =>  5 	 Predict =>  9
True =>  6 	 Predict =>  9
True =>  7 	 Predict =>  7
True =>  8 	 Predict =>  9
True =>  9 	 Predict =>  9
Predicting for  2000
--------------------------------------------------------------------
True =>  0 	 Predict =>  0
True =>  1 	 Predict =>  1
True =>  2 	 Predict =>  4
True =>  3 	 Predict =>  9
True =>  4 	 Predict =>  4
True =>  5 	 Predict =>  4
True =>  6 	 Predict =>  6
True